In [167]:
import tensorflow.keras as keras
import numpy as np

# Load the Fashion MNIST dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.fashion_mnist.load_data()

In [168]:
# Change the orientation of the dataset inorder to work

In [169]:
import numpy as np
from scipy.signal import correlate2d, convolve2d

class Convolution:

    def __init__(self, input_shape, filter_size, num_filters):
        input_height, input_width = input_shape
        self.num_filters = num_filters
        self.input_shape = input_shape

        # Size of outputs and filters or kernels or weigths

        self.filter_shape = (num_filters, filter_size, filter_size) # (3,3)
        self.output_shape = (num_filters, input_height - filter_size + 1, input_width - filter_size + 1)

        self.filters = np.random.randn(*self.filter_shape)
        self.biases = np.random.randn(*self.output_shape)


    def cross_correlation2D(self, input_mat, filter_mat):
        # Get the dimensions of the image and kernel
        image_height, image_width = input_mat.shape
        # (3, 3) shape
        kernel_height, kernel_width = filter_mat.shape

        # Calculate the output dimensions
        output_height = image_height - kernel_height + 1
        output_width = image_width - kernel_width + 1

        # Create an empty array to store the cross-correlation result
        result = np.zeros((output_height, output_width))

        # Perform cross-correlation
        for i in range(output_height):
            for j in range(output_width):
                patch = input_mat[i:i + kernel_height, j:j + kernel_width]
                result[i, j] = np.sum(patch * filter_mat)
        return result


    def forward(self, input_data):
        self.input_data = input_data
        # Initialized the input value
        output = np.zeros(self.output_shape)
        for i in range(self.num_filters):
            output[i] = correlate2d(self.input_data, self.filters[i], mode="valid")
        #Applying Relu Activtion function
        output = np.maximum(output, 0)
        return output


    def backward(self, dL_dout, lr):
        # Create a random dL_dout array
        dL_dinput = np.zeros_like(self.input_data)
        dL_dfilters = np.zeros_like(self.filters)

        for i in range(self.num_filters):
                # Calculating the gradient of the filters or weights
                dL_dfilters[i] = correlate2d(self.input_data, dL_dout[i],mode="valid")

                # Calculating the gradient of the inputs
                dL_dinput += convolve2d(dL_dout[i], self.filters[i], mode="full")

        self.filters -= lr * dL_dfilters
        self.biases -= lr * dL_dout


        return dL_dinput


class MaxPool:

    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward(self, input_data):
        self.input_data = input_data
        self.num_channels, self.input_height, self.input_width = input_data.shape
        self.output_height = self.input_height // self.pool_size
        self.output_width = self.input_width // self.pool_size

        self.output = np.zeros((self.num_channels, self.output_height, self.output_width))


        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size
                    patch = input_data[c, start_i:end_i, start_j:end_j]

                    self.output[c, i, j] = np.max(patch)

        return self.output

    def backward(self, dL_dout, lr):
        dL_dinput = np.zeros_like(self.input_data)

        for c in range(self.num_channels):
            for i in range(self.output_height):
                for j in range(self.output_width):
                    start_i = i * self.pool_size
                    start_j = j * self.pool_size

                    end_i = start_i + self.pool_size
                    end_j = start_j + self.pool_size
                    patch = self.input_data[c, start_i:end_i, start_j:end_j]

                    mask = patch == np.max(patch)

                    dL_dinput[c,start_i:end_i, start_j:end_j] = dL_dout[c, i, j] * mask

        return dL_dinput

class Fully_Connected:

    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        #self.ouptut_neurons = output_neurons
        self.weights = np.random.randn(output_size, self.input_size)
        #print(self.weights)
        self.biases = np.random.rand(output_size, 1)


    def softmax(self, z):
        # Shift the input values to avoid numerical instability
        shifted_z = z - np.max(z)
        # Compute the exponentiated values
        exp_values = np.exp(shifted_z)

        # Compute the sum of exponentiated values using log-sum-exp trick
        sum_exp_values = np.sum(exp_values, axis=0)
        log_sum_exp = np.log(sum_exp_values)

        # Compute the softmax probabilities
        probabilities = exp_values / sum_exp_values

        return probabilities


    def softmax_derivative(self, s):
        return np.diagflat(s) - np.dot(s, s.T)

    def forward(self, input_data):
        self.input_data = input_data
        flattened_input = input_data.flatten().reshape(1, -1)
        self.z = np.dot(self.weights, flattened_input.T) + self.biases
        self.output = self.softmax(self.z)
        return self.output

    def backward(self, dL_dout, lr):
        # Calculate the gradient of the loss with respect to the pre-activation (z)
        dL_dy = np.dot(self.softmax_derivative(self.output), dL_dout)
        # Calculate the gradient of the loss with respect to the weights (dw)
        dL_dw = np.dot(dL_dy, self.input_data.flatten().reshape(1, -1))

        # Calculate the gradient of the loss with respect to the biases (db)
        dL_db = dL_dy

        # Calculate the gradient of the loss with respect to the input data (dL_dinput)
        dL_dinput = np.dot(self.weights.T, dL_dy)
        dL_dinput = dL_dinput.reshape(self.input_data.shape)

        # Update the weights and biases based on the learning rate and gradients
        self.weights -= lr * dL_dw
        self.biases -= lr * dL_db

        # Return the gradient of the loss with respect to the input data
        return dL_dinput

In [184]:
X_train = train_images[:3000] / 255.0
y_train = train_labels[:3000]

X_test = train_images[3000:60000] / 255.0
y_test = train_labels[3000:60000]

X_train.shape

(3000, 28, 28)

In [185]:
from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

y_test[0]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)

In [172]:
conv = Convolution(X_train[0].shape, 6, 1)
pool = MaxPool(2)
full = Fully_Connected(121,10)


def cross_entropy_loss(predictions, targets):

    num_samples = 10

    # Avoid numerical instability by adding a small epsilon value
    epsilon = 1e-7
    predictions = np.clip(predictions, epsilon, 1 - epsilon)

    # Calculate the categorical cross-entropy loss
    loss = -np.sum(targets * np.log(predictions)) / num_samples

    return loss

def cross_entropy_loss_gradient(actual_labels, predicted_probs):
    num_samples = actual_labels.shape[0]

    # Calculate the gradient of the cross-entropy loss function
    gradient = -actual_labels / (predicted_probs + 1e-7) / num_samples

    return gradient


def predict(input_sample, conv, pool, full):
    # Forward pass through convolutional and pooling layers
    conv_out = conv.forward(input_sample)
    pool_out = pool.forward(conv_out)

    # Flatten the output feature maps
    flattened_output = pool_out.flatten()

    # Forward pass through fully connected layer
    predictions = full.forward(flattened_output)

    return predictions

def train_network(X, y, conv, pool, full, lr=0.01, epochs=200):
    for epoch in range(epochs):
        total_loss = 0.0
        correct_predictions = 0

        for i in range(len(X)):
            # Forward pass
            conv_out = conv.forward(X[i])
            pool_out = pool.forward(conv_out)
            full_out = full.forward(pool_out)

            # Calculate loss and accuracy
            loss = cross_entropy_loss(full_out.flatten(), y[i])
            total_loss += loss


            one_hot_pred = np.zeros_like(full_out)
            one_hot_pred[np.argmax(full_out)] = 1
            one_hot_pred = one_hot_pred.flatten()

            num_pred = np.argmax(one_hot_pred)
            num_y = np.argmax(y[i])

            if num_pred == num_y:
                correct_predictions += 1
            # Backward pass
            gradient = cross_entropy_loss_gradient(y[i], full_out.flatten()).reshape((-1, 1))
            full_back = full.backward(gradient, lr)
            pool_back = pool.backward(full_back, lr)
            conv_back = conv.backward(pool_back, lr)

        # Print epoch statistics
        average_loss = total_loss / len(X)
        accuracy = correct_predictions / len(X_train) * 100.0
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {average_loss:.4f} - Accuracy: {accuracy:.2f}%")


In [173]:
train_network(X_train, y_train, conv, pool, full)

Epoch 1/200 - Loss: 0.9825 - Accuracy: 18.37%
Epoch 2/200 - Loss: 0.5256 - Accuracy: 22.83%
Epoch 3/200 - Loss: 0.2406 - Accuracy: 29.53%
Epoch 4/200 - Loss: 0.1882 - Accuracy: 37.20%
Epoch 5/200 - Loss: 0.1693 - Accuracy: 42.37%
Epoch 6/200 - Loss: 0.1577 - Accuracy: 45.40%
Epoch 7/200 - Loss: 0.1485 - Accuracy: 48.83%
Epoch 8/200 - Loss: 0.1412 - Accuracy: 51.10%
Epoch 9/200 - Loss: 0.1349 - Accuracy: 53.50%
Epoch 10/200 - Loss: 0.1297 - Accuracy: 54.83%
Epoch 11/200 - Loss: 0.1254 - Accuracy: 56.90%
Epoch 12/200 - Loss: 0.1215 - Accuracy: 57.63%
Epoch 13/200 - Loss: 0.1181 - Accuracy: 59.00%
Epoch 14/200 - Loss: 0.1150 - Accuracy: 59.63%
Epoch 15/200 - Loss: 0.1124 - Accuracy: 60.37%
Epoch 16/200 - Loss: 0.1097 - Accuracy: 61.60%
Epoch 17/200 - Loss: 0.1076 - Accuracy: 61.87%
Epoch 18/200 - Loss: 0.1056 - Accuracy: 62.53%
Epoch 19/200 - Loss: 0.1038 - Accuracy: 63.47%
Epoch 20/200 - Loss: 0.1021 - Accuracy: 64.13%
Epoch 21/200 - Loss: 0.1004 - Accuracy: 64.57%
Epoch 22/200 - Loss: 0

In [186]:
predictions = []
for data in X_test:
    pred = predict(data, conv, pool, full)
    one_hot_pred = np.zeros_like(pred)
    one_hot_pred[np.argmax(pred)] = 1
    predictions.append(one_hot_pred.flatten())

predictions = np.array(predictions)

predictions

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [187]:
from sklearn.metrics import accuracy_score

accuracy_score(predictions, y_test)

0.7672456140350877

In [189]:
predictions.shape

(57000, 10)

In [136]:
y_test[:10]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [190]:
from sklearn.metrics import classification_report

print(classification_report(predictions, y_test))

              precision    recall  f1-score   support

           0       0.70      0.79      0.74      5093
           1       0.91      0.92      0.91      5645
           2       0.52      0.70      0.59      4244
           3       0.81      0.79      0.80      5904
           4       0.68      0.66      0.67      5891
           5       0.83      0.87      0.85      5473
           6       0.56      0.43      0.49      7405
           7       0.86      0.84      0.85      5765
           8       0.90      0.89      0.89      5764
           9       0.91      0.89      0.90      5816

   micro avg       0.77      0.77      0.77     57000
   macro avg       0.77      0.78      0.77     57000
weighted avg       0.77      0.77      0.77     57000
 samples avg       0.77      0.77      0.77     57000

